In [22]:
from time import sleep

import pyperclip as cb
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import win32com.client as win32

In [23]:
chromedriver_path = r"C:\Users\jjc\Desktop\한컴오피스 업무자동화 입문\chromedriver-win64\chromedriver.exe"
chrome_options = Options()
service = Service(executable_path=chromedriver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get('http://dart.fss.or.kr/dsac001/mainAll.do')

In [24]:
# onclick_list = []
# for i in driver.find_elements_by_css_selector('a[href^="/dsaf001/main.do?rcpNo="]'):
# onclick_list.append(i.get_attribute('onclick'))
onclick_list = [
i.get_attribute('onclick') for i in driver.find_elements(By.CSS_SELECTOR, 'a[href^="/dsaf001/main.do?rcpNo="]')]
onclick_list[:5]

["openReportViewer('20240827000227'); return false;",
 "openReportViewer('20240827000222'); return false;",
 "openReportViewer('20240827000221'); return false;",
 "openReportViewer('20240827000219'); return false;",
 "openReportViewer('20240827000215'); return false;"]

In [26]:
link_list = []
for i in onclick_list:
    # Execute the script to open the new window
    driver.execute_script(i)

    # Wait until a new window is opened
    WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)

    # Switch to the newly opened window
    driver.switch_to.window(driver.window_handles[1])

    # Wait until the download element is present
    download_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'a[href="#openPdfDownload"]'))
    )
    download_script = download_element.get_attribute('onclick')

    # Execute the 'onclick' script to open the download window
    driver.execute_script(download_script)

    # Wait until the new window for the PDF is opened
    WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 2)

    # Switch to the PDF download window
    driver.switch_to.window(driver.window_handles[2])

    # Wait until the PDF link is present and retrieve it
    pdf_link = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'a[href^="/pdf"]'))
    ).get_attribute('href')

    # Append the PDF link to the list
    current_len = len(link_list)
    link_list.append(pdf_link)

    # Wait until the list is updated
    while len(link_list) == current_len:
        sleep(0.1)
    print(link_list[-1])

    # Close the opened tabs and switch back to the original window
    driver.close()
    driver.switch_to.window(driver.window_handles[1])
    driver.close()
    driver.switch_to.window(driver.window_handles[0])


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF72496B632+29090]
	(No symbol) [0x00007FF7248DE6E9]
	(No symbol) [0x00007FF72479B1CA]
	(No symbol) [0x00007FF7247EEFD7]
	(No symbol) [0x00007FF7247EF22C]
	(No symbol) [0x00007FF7248397F7]
	(No symbol) [0x00007FF72481672F]
	(No symbol) [0x00007FF7248365D9]
	(No symbol) [0x00007FF724816493]
	(No symbol) [0x00007FF7247E09B1]
	(No symbol) [0x00007FF7247E1B11]
	GetHandleVerifier [0x00007FF724C8881D+3294093]
	GetHandleVerifier [0x00007FF724CD4403+3604339]
	GetHandleVerifier [0x00007FF724CCA2C7+3563063]
	GetHandleVerifier [0x00007FF724A26F16+797318]
	(No symbol) [0x00007FF7248E986F]
	(No symbol) [0x00007FF7248E5454]
	(No symbol) [0x00007FF7248E55E0]
	(No symbol) [0x00007FF7248D4A7F]
	BaseThreadInitThunk [0x00007FF910EB257D+29]
	RtlUserThreadStart [0x00007FF9127CAF28+40]


In [ ]:
# 너무 오래 걸리는 관계로.
import pickle
with open(r"C:WUsers#smj02#Desktopthwp-파이썬관련코드wlink_list.pickle", 'rb') as f:
    link_list = pickle.load(f)

In [ ]:
link_list [:3]

In [ ]:
# driver.page_source
# table = pd.read_html(driver.page_source)
table = pd.read_html (driver.page_source) [0]

driver.close()

In [ ]:
table.head()

In [ ]:
len(table)

In [ ]:
len(table.columns)

In [ ]:
hwp = win32.Dispatch( 'HWPFrame.HwpObject')
hwp.RegisterModule("FilePathCheckDLL", "SecurityModule") # 50/25

In [ ]:
hwp.Open(r"C:WUsersWsmj02WDesktopWdart_template.hwp")

In [ ]:
def hwp_insert_hyperlink(text, url):
    hwp.HAction.Run("TableCelIBlock")
    hwp.HAction.GetDefault("InsertHyperlink", hwp.HParameterSet.HHyperLink.HSet)
    hwp.HParameterSet.HHyperLink. Text = text
    hwp.HParameterSet.HHyperLink. Command = ur
    hwp.HAction.Execute("InsertHyperlink", hwp.HParameterSet.HHyperLink.HSet)

In [ ]:
def hwp_insert_text(text):
    hwp.HAction. GetDefault ("InsertText", hwp.HParameterSet .HInsertText .HSet)
    hwp.HParameterSet.HInsertText.Text = text
    hwp.HAction.Execute(" InsertText", hwp.HParameterSet.HInsertText.HSet)

In [ ]:
# for i in len(table):
#    for j in table. loc[i]:
#        cb.copy(j)
#        hwp.Aun('Paste')
#        hwp. Aun('TableRightCe/|Append'
#    hwp.Aun('TableAppendRow')

In [ ]:
# 종 더 다듬어보면
for i in range(len(table)):
    for idx, text in enumerate(table.loc[i]):
        if idx == 0 and i == 0:
            hwp_insert_text(text)
        elif idx == 2: # 보고서명 칼럼, 하이퍼링크를 삽입해야 한다.
            hwp.Run( 'TableRightCel lAppend')
            hwp_insert_hyperlink(text=text, url='{}:1:0;0'.format(link_list[i].replace("?","₩#?")))
        elif idx == 5: # 오른쪽 끝 비고란, 전부 Nan이므로 아무것도 입력하지 않고 넘어간다.
            hwp.Run('TableRightCel lAppend')
        else: # 그 외에는 텍스트를 그대로 입력한다.
            hwp.Run('TableRightCellAppend')
            hwp_insert_text(text)